### 1.0 Libraries and set-up

In [1]:
import ee
import os
import math
import geemap
import json
from shapely.geometry import shape
import geopandas as gpd

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
# region = 'YKD_weekly'
# region = 'AKCP_weekly'
# region = 'TUK_MKD_weekly'
region = 'YKF_weekly'
folder = 'projects/alpod-412314/assets/' + region

In [5]:
image_list = ee.data.listImages(folder)
image_list = image_list.get('images')

In [6]:
#print(image_list[0:5])
# rando_img = ee.Image(image_list[0]['id'])
# rando_img_stats = rando_img.reduceRegion(
#     reducer = ee.Reducer.minMax(),
#     scale = 10
# )

# rando_properties = rando_img.getInfo()['properties']
# rando_coords = rando_properties['system:footprint']['coordinates']
# print(rando_coords)
# polygon = ee.Geometry.Polygon(rando_coords)

### 2.0 Images for target time period

In [8]:
target_years = list(range(2019, 2024))
target_years = [str(year) for year in target_years]
print(target_years)

# June
#target_weeks = list(range(22, 27))
#August 
#target_weeks = list(range(31, 36))
# September
target_weeks = list(range(35, 40))


target_weeks = [str(week) for week in target_weeks]
print(target_weeks)

max_times = len(target_years) * len(target_weeks)
print(max_times)

['2019', '2020', '2021', '2022', '2023']
['35', '36', '37', '38', '39']
25


In [9]:
target_imgs = []
target_polygons = []

for image in image_list:
    img_id = image['id']
    year = img_id.split('_')[-2]
    week = img_id.split('_')[-1]

    if week in target_weeks and year in target_years:
        img_ee = ee.Image(img_id)
        img_properties = img_ee.getInfo()['properties']
        img_coords = img_properties['system:footprint']['coordinates']
        polygon_ee = ee.Geometry.Polygon(img_coords)
        
        target_imgs.append(img_ee)
        target_polygons.append(polygon_ee)

print(len(target_imgs))

150


### 3.0 Export the ROI foot print

In [10]:
p1 = target_polygons[0]
p2 = target_polygons[1]

combined_footprint = p1.union(p2)

for i in range(2, len(target_polygons)):
    p = target_polygons[i]
    combined_footprint = combined_footprint.union(p)

# combined_footprint_info = combined_footprint.getInfo()
# geojson_footprint = json.dumps(combined_footprint_info)
# geometry = shape(json.loads(geojson_footprint))
# roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}.shp')

### 4.0 Use Reducer to Aggregate for Time Interval

In [11]:
target_collection = ee.ImageCollection(target_imgs)
wtr_sum_img = target_collection.select('water_occurance').sum()
wtr_sum_img = wtr_sum_img.clip(combined_footprint)

### 5.0 Binary image based on > x number of occurances

In [12]:
time_fractions = [1/5, 1/6, 1/7, 1/10]
#print(time_fractions)
water_thresholds = [time_fraction * max_times for time_fraction in time_fractions]
print(water_thresholds)

binary_images = {}
for threshold in water_thresholds:
    threshold_round = math.floor(threshold)
    wtr_img_binary = wtr_sum_img.gt(threshold)
    binary_images[threshold_round] = wtr_img_binary

print(binary_images)

[5.0, 4.166666666666666, 3.571428571428571, 2.5]
{5: <ee.image.Image object at 0x1624138c0>, 4: <ee.image.Image object at 0x1624103b0>, 3: <ee.image.Image object at 0x162410920>, 2: <ee.image.Image object at 0x162412480>}


### 6.0 Visualize results

In [14]:
viz1 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 1,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map = geemap.Map(center=(65, -135), zoom=4)

occurance_cnts = binary_images.keys()
print(occurance_cnts)

for cnt in occurance_cnts:
    Map.addLayer(binary_images[cnt], viz1, f'{region}, {cnt}, {target_weeks[0]}')


#Map.addLayer(combined_footprint, {}, 'roi_bounds')
Map


dict_keys([5, 4, 3, 2])


Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### 7.1 Export the .GeoTiff to Drive

In [15]:
for cnt in occurance_cnts:

    img = binary_images[cnt]

    export_task = ee.batch.Export.image.toDrive(
        image=img,
        description=f'{region}_count{cnt}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
        folder='ewebb_s2',
        fileNamePrefix=f'{region}_count{cnt}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
        region=combined_footprint,
        scale=10,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    
    export_task.start()

### X.2 Export the .shp files to Drive??

### * Scrap workspace

In [ ]:
# binary_sum = weekly_sum.gt(3)

# vectors = binary_sum.reduceToVectors(
#     geometryType='polygon',
#     reducer=ee.Reducer.countEvery(),
#     scale=10,
#     maxPixels=1e13
# )
# # !!! Need to switch to a for-loop, becuase .getInfo() won't map to the server-side. 
# def get_geojson(vectors):
#     return ee.Feature(vectors).geometry().getInfo()

# vector_geoms = vectors.map(get_geojson)

# print(vector_geoms)

In [43]:
# np_img = geemap.ee_to_numpy(clipped_mosaic)

In [44]:
# t = target_collection.clip(combined_footprint)
# target_mosaic = target_collection.mosaic()
# clipped_mosaic = target_mosaic.clip(combined_footprint)

60
